In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# Data preprocessing

# Preprocess training set
# Perform image augmentation to prevent overfitting
train_datagen = ImageDataGenerator(rescale=1./255, # rescale performs feature scaling
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
# Connect to training set
# Convolutional neural network is trained and tested on batches of images
training_set = train_datagen.flow_from_directory('../dataset/training_set',
                                                   target_size=(64, 64),
                                                   batch_size=32, # number of images in each batch
                                                   class_mode='binary') # class_mode is either binary or categorical

test_datagen = ImageDataGenerator(rescale=1./255)

# Connect to test set
test_set = test_datagen.flow_from_directory('../dataset/test_set',
                                           target_size=(64, 64), # must be same format as training data
                                           batch_size=32,
                                           class_mode='binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [3]:
# Create convolutional neural network as sequence of layers
cnn = tf.keras.models.Sequential()

# Convolutional layer
# filters=no_of_feature_detectors, kernel_size=size_of_feature_detector(width/length), input_shape=(length, width, dim) (dim is 3 for colored images)
# strides is kept to default of (1, 1)
# input_shape is specified only for first convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Pooling layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=(2, 2)))

In [4]:
# Add second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=(2, 2)))

In [5]:
# Flattening
cnn.add(tf.keras.layers.Flatten())

In [6]:
# Full connection

# Add hidden layer
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

# Add output layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [7]:
# Compile convolutional neural network
# Use adam optimizer to perform stochastic gradient descent
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# Train convolutional neural network on training set and evaluate it on test set
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 [==============================] - 106s 423ms/step - loss: 0.6789 - accuracy: 0.5702 - val_loss: 0.6368 - val_accuracy: 0.6270
Epoch 2/25
250/250 [==============================] - 98s 392ms/step - loss: 0.6121 - accuracy: 0.6647 - val_loss: 0.5866 - val_accuracy: 0.6865
Epoch 3/25
250/250 [==============================] - 95s 379ms/step - loss: 0.5663 - accuracy: 0.7032 - val_loss: 0.5745 - val_accuracy: 0.7085
Epoch 4/25
250/250 [==============================] - 104s 417ms/step - loss: 0.5321 - accuracy: 0.7259 - val_loss: 0.5454 - val_accuracy: 0.7310
Epoch 5/25
250/250 [==============================] - 102s 409ms/step - loss: 0.5113 - accuracy: 0.7480 - val_loss: 0.5092 - val_accuracy: 0.7635
Epoch 6/25
250/250 [==============================] - 100s 401ms/step - loss: 0.4883 - accuracy: 0.7649 - val_loss: 0.4994 - val_accuracy: 0.7685
Epoch 7/25
250/250 [==============================] - 99s 394ms/step - loss: 0.4611 - accuracy: 0.7751 - val_loss: 0.4750 - va

In [11]:
# Make single prediction

import numpy as np
from keras.preprocessing import image

# Load image to predict
# Image format must be same as training/test data
test_image = image.load_img(path='../dataset/single_prediction/cat_or_dog_2.jpg', target_size=(64, 64))

# Convert image to 2D array to input into predict()
test_image = image.img_to_array(test_image)

# Image must be in batch so batch is added as extra (first) dimension
# Perform feature scaling on image
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0

result = cnn.predict(test_image)
# Check which binary output corresponds to which label
print(training_set.class_indices)

# First index refers to batch, second index refers to first (and only) image in that batch
if (result[0][0] > 0.5):
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)

{'cats': 0, 'dogs': 1}
cat
